In [1]:
# Standard scientific Python imports
import matplotlib.pyplot as plt

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics

# The digits dataset
digits = datasets.load_digits()

# The data that we are interested in is made of 8x8 images of digits, let's
# have a look at the first 4 images, stored in the `images` attribute of the
# dataset.  If we were working from image files, we could load them using
# matplotlib.pyplot.imread.  Note that each image must have the same size. For these
# images, we know which digit they represent: it is given in the 'target' of
# the dataset.

In [2]:
images_and_labels = list(zip(digits.images, digits.target))
for index, (image, label) in enumerate(images_and_labels[:4]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %i' % label)

In [20]:
# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
from keras.utils.np_utils import to_categorical
digits.target = to_categorical(digits.target)

n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
X_train = data[:n_samples // 2]
X_test = data[n_samples // 2:]
y_train = digits.target[:n_samples // 2]
y_test = digits.target[n_samples // 2:]

In [73]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from numpy.random import seed
seed(8)
model = Sequential()
model.add(Dense(20, input_shape=(data.shape[1],), activation='relu')) #first hidden layer, 20 relu nodes
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_split=0.3, epochs=50, callbacks=[EarlyStopping(patience=2)])

Train on 628 samples, validate on 270 samples
Epoch 1/50
628/628 [==============================] - 0s - loss: 3.5451 - acc: 0.0939 - val_loss: 2.8188 - val_acc: 0.1000
Epoch 2/50
628/628 [==============================] - 0s - loss: 2.2618 - acc: 0.1704 - val_loss: 2.2273 - val_acc: 0.1852
Epoch 3/50
628/628 [==============================] - 0s - loss: 1.8517 - acc: 0.2882 - val_loss: 2.0002 - val_acc: 0.2444
Epoch 4/50
628/628 [==============================] - 0s - loss: 1.5874 - acc: 0.3933 - val_loss: 1.8647 - val_acc: 0.3185
Epoch 5/50
628/628 [==============================] - 0s - loss: 1.3492 - acc: 0.5000 - val_loss: 1.7184 - val_acc: 0.4074
Epoch 6/50
628/628 [==============================] - 0s - loss: 1.1287 - acc: 0.6035 - val_loss: 1.6187 - val_acc: 0.4667
Epoch 7/50
628/628 [==============================] - 0s - loss: 0.9168 - acc: 0.7022 - val_loss: 1.4550 - val_acc: 0.5370
Epoch 8/50
628/628 [==============================] - 0s - loss: 0.7392 - acc: 0.7818 - val_l

In [74]:
predictions = model.predict(X_test)
# The classification report required 1D arrays, so I'll convert everything to max probability
from numpy import argmax
predictions1D = argmax(predictions, axis=1) #should have shape (899,)
y_test1D = argmax(y_test, axis=1)
report = metrics.classification_report(y_test1D, predictions1D)

import pandas as pd #for formatting
df = pd.DataFrame([x.split() for x in report.replace('/', '\n').split('\n')]) #have a problematic slash in there
from numpy import concatenate, array
df.columns = concatenate([array(['Digit']), df.iloc[0,:-1]])
df.drop(df.index[:2], inplace=True)
df.set_index('Digit', inplace=True)
df.drop(df.index[pd.isnull(df.index.values)], inplace=True)
df

,precision,recall,f1-score,support
Digit,,,,
0,0.99,0.95,0.97,88
1,0.92,0.75,0.82,91
2,0.91,0.91,0.91,86
3,0.92,0.84,0.87,91
4,0.98,0.95,0.96,92
5,0.87,0.79,0.83,91
6,0.82,0.98,0.89,91
7,0.90,0.84,0.87,89
8,0.69,0.75,0.72,88


This looks like it stabilized around 86% accuracy. 

In [77]:
sum(predictions1D == y_test1D)/len(y_test1D)

0.86651835372636266

This is much worse than, say, SVM, which got to 97%.